# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
print(len(file_path_list))

/home/workspace
2


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
for f in file_path_list:
    # reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object to iterate through the csv
        csvreader = csv.reader(csvfile)
        # will give you the next values of the iteration
        next(csvreader)  
        
 # extracting each data row one by one and append it to the empty list        
        for line in csvreader:
            full_data_rows_list.append(line) 
            
print(len(full_data_rows_list))

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    #writerow will create the column for the new csv
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                     'level','location','sessionId','song','userId'])
    #each row in the full_data_rows_list = data of an indivisual csv
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

394


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

334


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [6]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [7]:
# Create a Keyspace that will hold all the tables
session.execute("""
                CREATE KEYSPACE IF NOT EXISTS sparkify_cassandra
                WITH REPLICATION=
                {'class' : 'SimpleStrategy', 'replication_factor' : 1}
                """)

#### Set Keyspace

In [8]:
# connecting to the keyspace to use it as a session 
session.set_keyspace('sparkify_cassandra')

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [9]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

# Creating a table that will give the history of music playlist(artist name, song title and length), tables always tailored to match the query
query = """CREATE TABLE IF NOT EXISTS sparkify_cassandra.music_app_history (
               sessionId int,
               itemInSession int,
               artist text,
               song text,
               length float, 
           PRIMARY KEY(sessionId, itemInSession))"""
session.execute(query)
print(query)

CREATE TABLE IF NOT EXISTS sparkify_cassandra.music_app_history (
               sessionId int,
               itemInSession int,
               artist text,
               song text,
               length float, 
           PRIMARY KEY(sessionId, itemInSession))


In [10]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    # looping through the csv file and inserting every row under the column created (sessionId, itermInSession...)
    for line in csvreader:
        # print(line)
        query = "INSERT INTO music_app_history (sessionId, itemInSession, artist, song, length)"
        query = query + "VALUES (%s,%s,%s,%s,%s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0],line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [12]:
# Since dataset given doesn't accomedate the queries, comment out the extra query written below to ensure the data model, insertion is accurate
#query = "SELECT artist, length, song FROM music_app_history WHERE sessionId = 139 AND itemInSession = 3"
query = "SELECT artist, length, song FROM music_app_history WHERE sessionId = 338 AND itemInSession = 4"
rows = session.execute(query)
rows = list(rows)

if rows == []:
    print("There is no data that matches the given query",rows)
#converting query output into pandas dataframe to see the result printed in table
query1 =pd.DataFrame(rows) 
query1.head()

# to see what each column holds from data 
#for row in rows:
#    print(row)

There is no data that matches the given query []


""


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [16]:
#Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
query = """CREATE TABLE IF NOT EXISTS sparkify_cassandra.song_playlist_session (
               sessionId int,
               itemInSession int,
               userId int,
               artist text,
               song text,
               firstName text,
               lastName text,
           PRIMARY KEY((userId, sessionId), itemInSession))"""
session.execute(query)
print(query)            

CREATE TABLE IF NOT EXISTS sparkify_cassandra.song_playlist_session (
               sessionId int,
               itemInSession int,
               userId int,
               artist text,
               song text,
               firstName text,
               lastName text,
           PRIMARY KEY((userId, sessionId), itemInSession))


In [17]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header

    for line in csvreader:
        query = "INSERT INTO song_playlist_session (sessionId, itemInSession, userId, artist, song, firstName, lastName)"
        query = query + "VALUES (%s,%s,%s,%s,%s,%s,%s)"
        session.execute(query, (int(line[8]), int(line[3]), int(line[10]),line[0], line[9], line[1], line[4]))

query = "SELECT itemInSession, artist, song, firstName, lastName FROM song_playlist_session WHERE userId = 10 AND sessionId = 182"
rows = session.execute(query)
rows = list(rows)
# print(rows)
if rows == []:
    print("There is no data that matches the given query",rows)
#converting query output into pandas dataframe to see the result printed in table
query1 =pd.DataFrame(rows) 
query1.head()

# to see what each column holds from data 
#for row in rows:
#    print(row)

There is no data that matches the given query []


""


In [18]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
query = """
           CREATE TABLE IF NOT EXISTS sparkify_cassandra.famous_song_listened (
               song text, 
               userId text, 
               sessionId varint, 
               itemInSession varint, 
               firstName text, 
               lastName text,  
           PRIMARY KEY (song, userId))"""
session.execute(query)
print(query)


           CREATE TABLE IF NOT EXISTS sparkify_cassandra.famous_song_listened (
               song text, 
               userId text, 
               sessionId varint, 
               itemInSession varint, 
               firstName text, 
               lastName text,  
           PRIMARY KEY (song, userId))


In [19]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO famous_song_listened (song, userId, sessionId, itemInSession, firstName, lastName)"
        query = query + "VALUES (%s,%s,%s,%s,%s,%s)"
        session.execute(query, (line[8], line[10], int(line[8]),int(line[3]), line[1], line[4]))

query = "SELECT song, firstName, lastName FROM famous_song_listened WHERE song='All Hands Against His Own'"
rows = session.execute(query)
rows = list(rows)
# print(rows)
if rows == []:
    print("There is no data that matches the given query",rows)
#converting query output into pandas dataframe to see the result printed in table
query1 =pd.DataFrame(rows) 
query1.head()

There is no data that matches the given query []


""


### Drop the tables before closing out the sessions

In [4]:
## TO-DO: Drop the table before closing out the sessions

In [ ]:
session.execute("DROP TABLE music_app_1")
session.execute("DROP TABLE music_app_2")
session.execute("DROP TABLE famous_song_listened")


### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()